In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

In [ ]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))
session = driver.session()

gds = GraphDataScience(uri, auth=("neo4j", "password"))

In [ ]:
# Ingest data into neo4j 
gds.run_cypher('''LOAD CSV WITH HEADERS FROM 'file:///fraudTestSample.csv' AS row
    WITH row

    MERGE (p:Person {first: row.first, last: row.last})
    MERGE (m:Merchant {merchantName: row.merchant, isFraud: row.merch_is_fraud})
    MERGE (c:Category {categoryType: row.category})
    MERGE (txn:Transaction {txnID: row.trans_num, amount: toFloat(row.amt), timestamp: row.trans_date_trans_time})
    MERGE (cc:CreditCard {ccNum: row.cc_num})
    MERGE (personLoc:Geocode {latitude: toFloat(row.lat), longitude: toFloat(row.long)})
    MERGE (zip: Zipcode {code: row.zip})
    MERGE (city: City {name: row.city})
    MERGE (state: State {name: row.state})
    MERGE (merchantLoc:Geocode {latitude: toFloat(row.merch_lat), longitude: toFloat(row.merch_long)})
    MERGE (merch_zip: Zipcode {code: row.merch_zip_code})
    MERGE (merch_city: City {name: row.merch_po_name})
    MERGE (merch_state: State {name: row.merch_state})


    MERGE (p)-[:OWNS]->(cc)
    MERGE (cc)-[:MADE_A]->(txn)
    MERGE (txn)-[:PAID]->(m)
    MERGE (m)-[:IS_A]->(c)
    MERGE (m)-[:LOCATED_AT]->(merchantLoc)
    MERGE (p)-[:LOCATED_AT]->(personLoc)
    MERGE (personLoc)-[:IS_IN]->(zip)
    MERGE (zip)-[:IS_IN]->(city)
    MERGE (city)-[:IS_IN]->(state)
    MERGE (merchantLoc)-[:IS_IN]->(merch_zip)
    MERGE (merch_zip)-[:IS_IN]->(merch_city)
    MERGE (merch_city)-[:IS_IN]->(merch_state);''')

In [ ]:
# Show stats about the ingested data
gds.run_cypher("""
CALL apoc.meta.stats()
    YIELD labels
    UNWIND keys(labels) AS nodeLabel
    RETURN nodeLabel, labels[nodeLabel] AS nodeCount
""")